In [ ]:
import socket
import numpy as np
import cv2 
import threading

sckt = socket.socket()
sckt.bind(("", 1234))
sckt.listen()
session, address = sckt.accept()
print(session.recv(2046)) 
cameraIndex = 'http://192.168.0.101:8080/video'
camera = cv2.VideoCapture(cameraIndex) 

def sender():
    while True:
        status, photo = camera.read()
        photo = cv2.resize(photo, (640, 480))
        print(photo.shape)
        if status:
            session.send(np.ndarray.tobytes(photo))
        else: print("Not in frame")

def receiver():
    frames_error = 0
    print("Entered")
    while True:
        frames += 1
        data = session.recv(10000000000)
        if(data == b'finished'):
            print("Finished")
            camera.release()
            session.close()
        else:
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3:
                cv2.imshow('From Client side', photo.reshape(480, 640, 3))
                if cv2.waitKey(1) == 13:
                    session.send(b'finished')
                    camera.release()
                    cv2.destroyAllWindows()
                    break
            else:
                print("Unavailable {} frames".format(frames_error) )
                
threading.Thread(target=sender).start()
threading.Thread(target=receiver).start()